In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [3]:
# 2. Mount Drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import numpy as np
from PIL import Image,ImageEnhance
import cv2

from google.colab.patches import cv2_imshow

In [5]:
def find_nearest_above(my_array, target):
    diff = my_array - target
    mask = np.ma.less_equal(diff, -1)
    # We need to mask the negative differences
    # since we are looking for values above
    if np.all(mask):
        c = np.abs(diff).argmin()
        return c # returns min index of the nearest if target is greater than any value
    masked_diff = np.ma.masked_array(diff, mask)
    return masked_diff.argmin()

In [6]:
def hist_match(original, specified):

    oldshape = original.shape
    original = original.ravel()
    specified = specified.ravel()

    # get the set of unique pixel values and their corresponding indices and counts
    s_values, bin_idx, s_counts = np.unique(original, return_inverse=True,return_counts=True)
    t_values, t_counts = np.unique(specified, return_counts=True)

    # Calculate s_k for original image
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    
    # Calculate s_k for specified image
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # Round the values
    sour = np.around(s_quantiles*255)
    temp = np.around(t_quantiles*255)
    
    # Map the rounded values
    b=[]
    for data in sour[:]:
        b.append(find_nearest_above(temp,data))
    b= np.array(b,dtype='uint8')

    return b[bin_idx].reshape(oldshape)

In [7]:
def enhancer(im,a):
    #im = Image.open("drive/My Drive/Results 3/result0244 enhanced-a=0.6,b=0.5,g=-0.05,d=0.05 .JPG")
    enhancer = ImageEnhance.Brightness(im)
    enhanced_im = enhancer.enhance(a)
    enhanced_im.save("drive/My Drive/enhance/enhanced_sample.JPG")
    return 


In [23]:
def brighten(path,alpha,beta,gama,delta,psi,g):
    temp = cv2.imread(path)
    #im = Image.open(path)
    #Img2 = cv2.imread("drive/My Drive/enhance/histeq.JPG")
    Img1 = cv2.imread("drive/My Drive/Images_for_Adarsh/DSC_6139.JPG")
    Img2 = hist_match(temp,Img1)
    #cv2_imshow(Img1)
    hsv1 = cv2.cvtColor(Img1, cv2.COLOR_BGR2HSV)
    
    hsv2 = cv2.cvtColor(Img2, cv2.COLOR_BGR2HSV)
    l = hsv1.shape[0]
    b = hsv1.shape[1]
    diffv = hsv1 - hsv2
    diffv = np.array(diffv)
    sumb = np.sum(Img2[:,:,0])
    sumv = np.sum(np.square(diffv[:,:,2]))
    sums = np.sum(np.square(diffv[:,:,1]))
    sumh = np.sum(np.square(diffv[:,:,0]))
    #sumv = np.sum(diffv[:,:,2])
    #sums = np.sum(diffv[:,:,1])
    #sumh = np.sum(diffv[:,:,0])

    count = l*b
    a = np.sqrt((sumv/count))               #to find average difference in intensity('V'value)
    x = sumb/count                       #to find average blue component in current image
    s = np.sqrt((sums/count))               #to find average difference in saturation
    h = np.sqrt((sumh/count))               #to find average difference in hue
    #a = sumv/count
    #s = sums/count
    #h = sumh/count

    #final1 = cv2.cvtColor(hsv2, cv2.COLOR_HSV2BGR)
    """finalhist = histequal(Img2)
    temp = cv2.cvtColor(finalhist, cv2.COLOR_BGR2HSV)"""
    #hsv2[i][j][2] = np.minimum(255, np.maximum((alpha*hsv2[i][j][2]) + (beta*a), 0)) #adjusting the intensity value

    """for i in range(l):
        print(i)
        for j in range(b):
            temp[i][j][2] = np.minimum(255, np.maximum((alpha*temp[i][j][2]) + (beta*a), 0))      #adjusting the intensity value
            temp[i][j][1] = temp[i][j][1] + (gama*s)                                              #adjusting the saturation value
            temp[i][j][1] = temp[i][j][1] + (delta*h)                                             #adjusting the hue value"""


    print("Intensity,Saturation,Hue Adjustment")
    p = np.minimum(255, np.maximum((alpha*hsv2[:,:,2]) + (beta*a), 0))
    hsv2[:,:,2] = p
    q = 1.0*hsv2[:,:,1] + (gama*s)
    hsv2[:,:,1] = q
    r = (delta*hsv2[:,:,0]) + (psi*h)
    hsv2[:,:,0] = r
    final1 = cv2.cvtColor(hsv2, cv2.COLOR_HSV2BGR)
    cv2.imwrite("drive/My Drive/enhance/enhanced_sample.JPG",final1)
    im2 = Image.open("drive/My Drive/enhance/enhanced_sample.JPG")
    enhancer(im2,g)
    final = cv2.imread("drive/My Drive/enhance/enhanced_sample.JPG")
    print("Sky Unchanged")
    for i in range(l):
        print(i)
        for j in range(b):
            if temp[i][j][0] > (0.75*x):  
                  final[i][j] = temp[i][j]          #keeping the pixel values in sky part unchanged
    #hsvf = cv2.cvtColor(final1, cv2.COLOR_BGR2HSV) 
    #v = 1.0*hsvf[:,:,0] + (delta*h) 
    #hsvf[:,:,0] = v
    #final =  cv2.cvtColor(hsvf, cv2.COLOR_HSV2BGR)       
    
    cv2.imwrite("drive/My Drive/RESULTS FINAL/result0244 enhanced1 -a=1,b=1,g=0.05,d=0.5,psi=1 enh1.5.JPG", final)


In [24]:
def histequal(Img1):
    temp = cv2.cvtColor(Img1, cv2.COLOR_BGR2YUV)
    temp[:,:,0] = cv2.equalizeHist(temp[:,:,0])
    histeq = cv2.cvtColor(temp, cv2.COLOR_YUV2BGR)
    cv2.imwrite("drive/My Drive/enhance/histeq.JPG", histeq)
    return
    #cv2.imwrite("drive/My Drive/Results final2/equalized result0244-a=1,b=0.75,g=0.1,d=0.1.JPG", histeq)

In [ ]:
brighten("drive/My Drive/Images_for_Adarsh/DSC_0244.JPG",1,1,0.05,0.5,1,1.5)